## Copyright notice from VGG-Face
MIT License

Copyright (c) 2017 kbehouse

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Imports

In [ ]:
from os import listdir
from os.path import isdir
from random import choice

#Architecture and pretrained weights from https://github.com/serengil/tensorflow-101/blob/master/python/vgg-face.ipynb
import cv2
import numpy as np
from PIL import Image
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
from keras.models import Sequential
from matplotlib import pyplot
from numpy import asarray, savez_compressed, load, expand_dims
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.svm import SVC

# Extract faces from dataset

In [ ]:
FILENAME_DATASET = 'labs-dataset.npz'
FILENAME_FEATURE_VECTORS = 'labs-embeddings.npz'


def extract_face(filename, required_size=(224, 224)):
    img = Image.open(filename)
    img = img.resize(required_size)
    img_array = asarray(img)
    return img_array


# load images and extract faces for all images in a directory
def load_faces(directory):
    face_list = list()
    for filename in listdir(directory):
        if filename == '.DS_Store':
            continue
        path = directory + filename
        face = extract_face(path)
        face_list.append(face)
    return face_list


# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        path = directory + subdir + '/'
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        labels = [subdir for _ in range(len(faces))]
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)


# load train dataset
train_X, train_y = load_dataset('../faces/train/')
# load test dataset
test_X, test_y = load_dataset('../faces/val/')
# save arrays to one file in compressed format
savez_compressed(FILENAME_DATASET, train_X, train_y, test_X, test_y)

In [ ]:
#download the weights and put them in the same folder as this code from: https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))
model.load_weights('vgg_face_weights.h5')
model.compile()

In [ ]:
model.summary()

In [ ]:
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    sample = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(sample)
    return yhat[0]

In [ ]:
def load_compressed_dataset(path):
    # Load a dataset with np.load with pickle enabled and then put the normal np.load back
    # save np.load
    np_load_old = np.load

    # modify the default parameters of np.load
    np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True, **k)

    data = np.load(path)
    # putback the old load functionality
    load = np_load_old
    np.load = np_load_old
    return data


data = load_compressed_dataset(FILENAME_DATASET)

train_X, train_y, test_X, test_y = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', train_X.shape, train_y.shape, test_X.shape, test_y.shape)


def get_embeddingsarray(dataset):
    new_dataset = list()
    for face_pixels in dataset:
        embedding = get_embedding(model, face_pixels)
        new_dataset.append(embedding)
    return asarray(new_dataset)


# convert each face in train set to an embedding
new_train_X = get_embeddingsarray(train_X)
# do the same for the test set
new_test_X = get_embeddingsarray(test_X)
# save arrays to one file in compressed format
savez_compressed(FILENAME_FEATURE_VECTORS, new_train_X, train_y, new_test_X, test_y)

# Train SVC model

In [ ]:
# load faces
data = load(FILENAME_DATASET)
test_X_faces = data['arr_2']
# load face embeddings
data = load(FILENAME_FEATURE_VECTORS)
train_X, train_y, test_X, test_y = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# normalize input vectors
in_encoder = Normalizer(norm='l2')
train_X = in_encoder.transform(train_X)
print(train_X.shape)
test_X = in_encoder.transform(test_X)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(train_y)
train_y = out_encoder.transform(train_y)
test_y = out_encoder.transform(test_y)
# fit model
model_svc = SVC(kernel='linear', probability=True, C=3)
model_svc.fit(train_X, train_y)

# Test model

In [ ]:
selection = choice([i for i in range(test_X.shape[0])])
random_face_pixels = test_X_faces[selection]
random_face_emb = test_X[selection]
random_face_class = test_y[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])
# prediction for the face
samples = expand_dims(random_face_emb, axis=0)
yhat_class = model_svc.predict(samples)
yhat_prob = model_svc.predict_proba(samples)
# get name
class_probability = np.amax(yhat_prob)
index = np.where(yhat_prob == class_probability)
predict_names = out_encoder.classes_
print('Predicted: %s (%.3f)' % (predict_names[index[1]], class_probability))
print('Expected: %s' % random_face_name)
# plot for fun
pyplot.imshow(random_face_pixels)
title = '%s (%.3f)' % (predict_names[index[1]], class_probability)
pyplot.title(title)
pyplot.show()

# Read an image and get a prediction

In [ ]:
modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
imgLoc = '../banaan.png'
img = cv2.imread(imgLoc)
height, width = img.shape[:2]
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)),
                             1.0, (300, 300), (104.0, 117.0, 123.0))
net.setInput(blob)
faces = net.forward()
#OPENCV DNN
for i in range(faces.shape[2]):
    confidence = faces[0, 0, i, 2]
    if confidence > 0.5:
        box = faces[0, 0, i, 3:7] * np.array([width, height, width, height])
        faceimg = Image.fromarray(box)
        (x, y, x1, y1) = box.astype("int")

img = Image.open(imgLoc)
#img = img.crop((x , y, x1, y1))
img = img.resize((224, 224))
img = img.convert('RGB')
pyplot.imshow(img)
array = asarray(img)
embed = get_embedding(model, array)
embed = expand_dims(embed, axis=0)
in_encoder = Normalizer(norm='l2')
train_X = in_encoder.transform(embed)
yhat_class = model_svc.predict(train_X)
yhat_prob = model_svc.predict_proba(train_X)
print(yhat_prob)
class_probability = np.amax(yhat_prob)
index = np.where(yhat_prob == class_probability)
predict_names = out_encoder.classes_
print(predict_names)
print('Predicted: %s (%.3f)' % (predict_names[index[1]], class_probability))

# Setting up the Tensorboard projector

In [ ]:
import os
import shutil
from tensorboard.plugins import projector
import tensorflow as tf

In [ ]:
def plot_to_projector(
        x,
        feature_vector,
        y,
        class_names,
        log_dir="logsVGG",
        meta_file="metadata.tsv",
):
    assert x.ndim == 4  # (BATCH, H, W, C)

    if os.path.isdir(log_dir):
        shutil.rmtree(log_dir)

    # Create a new clean fresh folder :)
    os.mkdir(log_dir)

    sprites_file = os.path.join(log_dir, "sprites.png")
    sprite_x = x * 255
    sprite = create_sprite(sprite_x)
    img = Image.fromarray(sprite)
    img.save(sprites_file)

    # Generate label names
    labels = []
    for i in range(int(y.shape[0])):
        labels.append([])
        labels[i].append(class_names[y[i]])

    with open(os.path.join(log_dir, meta_file), "w") as f:
        for label in labels:
            f.write("{}\n".format(label))

    if feature_vector.ndim != 2:
        print(
            "NOTE: Feature vector is not of form (BATCH, FEATURES)"
            " reshaping to try and get it to this form!"
        )
        feature_vector = tf.reshape(feature_vector, [feature_vector.shape[0], -1])
    print(feature_vector.shape)

    feature_vector = tf.Variable(feature_vector)
    checkpoint = tf.train.Checkpoint(embedding=feature_vector)
    checkpoint.save(os.path.join(log_dir, "embeddings.ckpt"))

    # Set up config
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = meta_file
    embedding.sprite.image_path = "sprites.png"
    embedding.sprite.single_image_dim.extend((x.shape[1], x.shape[2]))
    projector.visualize_embeddings(log_dir, config)

In [ ]:
def create_sprite(data):
    """Creates the sprite image along with any necessary padding

    Args:
      data: NxHxW[x3] tensor containing the images.

    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[..., np.newaxis], (1, 1, 1, 3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1, 2, 3, 0) - min).transpose(3, 0, 1, 2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1, 2, 3, 0) / max).transpose(3, 0, 1, 2)
    data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
               (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
                  constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
                                                           + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
train_X, train_y = load_dataset('../faces/train/')
test_X, test_y = load_dataset('../faces/val/')

In [ ]:
data = load(FILENAME_FEATURE_VECTORS)
new_train_X, new_train_y, new_test_X, new_test_y = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# normalize input vectors
in_encoder = Normalizer(norm='l2')
new_train_X = in_encoder.transform(new_train_X)
new_test_X = in_encoder.transform(new_test_X)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(train_y)
new_train_y = out_encoder.transform(new_train_y)
new_test_y = out_encoder.transform(new_test_y)
X = np.concatenate((new_train_X, new_test_X), axis=0)
y = np.concatenate((new_train_y, new_test_y), axis=0)
new_X = []
for i in X:
    emb = np.zeros((1, 5))
    i = expand_dims(i, axis=0)
    emb[0, :] = model_svc.predict_proba(i)
    new_X.append(emb)
new_X = asarray(new_X)
new_X = np.rollaxis(new_X, 1, 0)
new_X = new_X[0, :, :]
X = np.concatenate((train_X, test_X), axis=0)

In [ ]:
plot_to_projector(X, new_X, asarray(y), ['Albert', 'Eline', 'Erik', 'Kevin', 'Marcel'])